In [7]:
#aggregated insurance

import os
import json
import pandas as pd
import psycopg2

path1 = "C:/Users/Knodez Guest/Desktop/PhonePe(P)/pulse/data/aggregated/insurance/country/india/state/"
agg_state_list = os.listdir(path1)
#Yes, 'path1' is a string, but it represents a real folder path in the local system. We use it in an imported os function called 'os.listdir()' to access/interact with the actual files and folders in the local system. Also, 'os.listdir()' function gives all the states in the '/state/' folder as a list.
#When you paste this path in your File Explorer you land directly inside the "state" folder, and what you see are all the state folders (like tamil-nadu, karnataka, etc.)
#But here's the key difference between you using File Explorer manually vs. Python using code: Python doesn't "see" anything until you ask it to list the contents using:
#This line tells Python:
#“Hey, go to this folder, and give me the names of all items (like state folders) inside it.”

columns1 = {"States":[],"Years":[],"Quarter":[],"Transaction_Type":[],"Transaction_Count":[],"Transaction_Amount":[]}

for state in agg_state_list:
    state_path = path1 + state + "/" #C:/Users/Knodez Guest/Desktop/PhonePe(P)/pulse/data/aggregated/insurance/country/india/state/andaman-&-nicobar-islands/ - #This loop goes through each state folder name in 'agg_state_list' [<- it is a list] and creates the full path to that state’s directory, storing it in 'state_path'.
    agg_year_list = os.listdir(state_path) #You're using agg_year_list = os.listdir(state_path) to get all year folders inside each state folder so you can process them next.
    
    for year in agg_year_list:
        year_path = state_path + year + "/" #C:/Users/Knodez Guest/Desktop/PhonePe(P)/pulse/data/aggregated/insurance/country/india/state/andaman-&-nicobar-islands/2020/ - #This loop goes through each year folder name in 'agg_year_list' [<- it is a list] and creates the full path to that year’s directory, storing it in 'year_path'.
        agg_file_list = os.listdir(year_path) #You're using agg_file_list = os.listdir(year_path)) to get all file folders inside each year folder so you can process them next.
        
        for file in agg_file_list:
            file_path = year_path + file #C:/Users/Knodez Guest/Desktop/PhonePe(P)/pulse/data/aggregated/insurance/country/india/state/andaman-&-nicobar-islands/2020/ - #This loop goes through each files name in 'agg_file_list' [<- it is a list] and creates the full path to that year’s directory, storing it in 'file_path'.
            
            data=open(file_path,"r") #Even though file_path is a string, the open() function understands it as a path. If the file exists at that location, Python gives you a file object (like a door to look inside the file).
            A = json.load(data) #JSON files are text files in a special format. json.load() reads that text and parses it into Python data types. This tells Python: “Read the contents of this file and convert it into a Python dictionary or list, based on the JSON structure.”
            
            for i in A["data"]["transactionData"]:
                name=i["name"]
                count = i["paymentInstruments"][0]["count"]
                amount = i["paymentInstruments"][0]["amount"]
                columns1["States"].append(state)
                columns1["Years"].append(year)
                columns1["Quarter"].append(int(file.strip(".json")))
                columns1["Transaction_Type"].append(name)
                columns1["Transaction_Count"].append(count)
                columns1["Transaction_Amount"].append(amount)     
                
aggregated_insurance = pd.DataFrame(columns1)

aggregated_insurance["States"] = aggregated_insurance["States"].str.replace("-"," ")
aggregated_insurance["States"] = aggregated_insurance["States"].str.title()

connection = psycopg2.connect(host="localhost",port="5432",user="postgres",password="12345",database="PhonePe_DB")
cursor = connection.cursor()
    
ai_create_q1 = "create table if not exists aggregated_insurance(states varchar(255), years int, quarter int, transaction_type varchar(255), transaction_count bigint, transaction_amount bigint)"
cursor.execute(ai_create_q1)
connection.commit()

ai_insert_q1 = "insert into aggregated_insurance(states, years, quarter, transaction_type, transaction_count, transaction_amount) values (%s,%s,%s,%s,%s,%s)"
values = aggregated_insurance.values.tolist()
cursor.executemany(ai_insert_q1,values)
connection.commit()  


In [ ]:
#aggregated_transaction

import os
import json
import pandas as pd
import psycopg2

columns2 = {"States":[],"Years":[],"Quarter":[],"Transaction_Type":[],"Transaction_Count":[],"Transaction_Amount":[]}

path2="C:/Users/Knodez Guest/Desktop/PhonePe(P)/pulse/data/aggregated/transaction/country/india/state/"
agg_state_list=os.listdir(path2)

for state in agg_state_list:
    state_path=path2+state+"/"
    agg_years_list=os.listdir(state_path)
    
    for year in agg_years_list:
        year_path=state_path+year+"/"
        agg_file_list=os.listdir(year_path)
        
        for file in agg_file_list:
            file_path=year_path+file
            
            data=open(file_path,"r")
            B=json.load(data)

            for i in B["data"]["transactionData"]:
                name=i["name"]
                count = i["paymentInstruments"][0]["count"]
                amount = i["paymentInstruments"][0]["amount"]
                
                columns2["Transaction_Type"].append(name)
                columns2["Transaction_Count"].append(count)
                columns2["Transaction_Amount"].append(amount)
                columns2["States"].append(state)
                columns2["Years"].append(year)
                columns2["Quarter"].append(int(file.strip(".json")))

aggregated_transaction = pd.DataFrame(columns2)

aggregated_transaction["States"] = aggregated_transaction["States"].str.replace("-"," ")
aggregated_transaction["States"] = aggregated_transaction["States"].str.title()

connection = psycopg2.connect(host="localhost",port="5432",user="postgres",password="12345",database="PhonePe_DB")
cursor = connection.cursor()

at_create_q1 = """create table if not exists aggregated_transaction(states varchar(255), years int, quarter int, transaction_type varchar(255), transaction_count bigint, transaction_amount bigint)"""
cursor.execute(at_create_q1)
connection.commit()

at_insert_q1 = """insert into aggregated_transaction(states, years, quarter, transaction_type, transaction_count, transaction_amount) values (%s,%s,%s,%s,%s,%s)"""
values = aggregated_transaction.values.tolist()
cursor.executemany(at_insert_q1,values)
connection.commit() 

In [11]:
#aggregated_user

import os
import json
import pandas as pd
import psycopg2

columns3={"States":[],"Years":[],"Quarter":[],"Brands":[],"Transaction_Count":[],"Percentage":[]}

path3="C:/Users/Knodez Guest/Desktop/PhonePe(P)/pulse/data/aggregated/user/country/india/state/"
agg_state_list=os.listdir(path3)

for state in agg_state_list:
    state_path=path3+state+"/"
    agg_year_list=os.listdir(state_path)
    
    for year in agg_year_list:
        year_path=state_path+year+"/"
        agg_file_path=os.listdir(year_path)
        
        for file in agg_file_path:
            file_path=year_path+file

            data=open(file_path,"r")
            C=json.load(data)
   
            try:
                for i in C["data"]["usersByDevice"]:
                    brand=i["brand"]
                    count = i["count"]
                    percentage = i["percentage"]
                    columns3["Brands"].append(brand)
                    columns3["Transaction_Count"].append(count)
                    columns3["Percentage"].append(percentage)
                    columns3["States"].append(state)
                    columns3["Years"].append(year)
                    columns3["Quarter"].append(int(file.strip(".json")))
            except:
                pass

aggregated_user = pd.DataFrame(columns3)

aggregated_user["States"] = aggregated_user["States"].str.replace("-"," ")
aggregated_user["States"] = aggregated_user["States"].str.title()
 
connection = psycopg2.connect(host = "localhost",port = "5432",user = "postgres",password = "12345",database = "PhonePe_DB")
cursor = connection.cursor()

au_create_q1 = """create table if not exists aggregated_user(states varchar(255), years int, quarter int, brands varchar(255), transaction_count bigint, percentage bigint)"""
cursor.execute(au_create_q1)
connection.commit()

au_insert_q1 = """insert into aggregated_user (states, years, quarter, brands, transaction_count, percentage) values (%s,%s,%s,%s,%s,%s)"""
values = aggregated_user.values.tolist()
cursor.executemany(au_insert_q1, values)
connection.commit()

In [ ]:
#map insurance

import os
import json
import pandas as pd
import psycopg2

columns4 = {"States":[],"Years":[],"Quarter":[],"Districts":[],"Transaction_Count":[],"Transaction_Amount":[]}

path4 = "C:/Users/Knodez Guest/Desktop/PhonePe(P)/pulse/data/map/insurance/hover/country/india/state/"
map_state_list = os.listdir(path4)

for state in map_state_list:
    state_path = path4 + state + "/"
    map_year_path = os.listdir(state_path)
    
    for year in map_year_path:
        year_path = state_path + year + "/"
        map_file_path = os.listdir(year_path)
        
        for file in map_file_path:
            file_path = year_path + file
            
            data = open(file_path,"r")
            D = json.load(data)

            for i in D["data"]["hoverDataList"]:
                name = i["name"]
                count = i["metric"][0]["count"]
                amount = i["metric"][0]["amount"]
                columns4["States"].append(state)
                columns4["Years"].append(year)
                columns4["Quarter"].append(int(file.strip(".json")))
                columns4["Districts"].append(name)
                columns4["Transaction_Count"].append(count)
                columns4["Transaction_Amount"].append(amount)

map_insurance = pd.DataFrame(columns4)

map_insurance["States"] = map_insurance["States"].str.replace("-"," ")
map_insurance["States"] = map_insurance["States"].str.title()
map_insurance["Districts"] = map_insurance["Districts"].str.replace("-"," ")
map_insurance["Districts"] = map_insurance["Districts"].str.title()

connection = psycopg2.connect(host="localhost",port="5432",user="postgres",password="12345",database="PhonePe_DB")
cursor = connection.cursor()
      
mi_create_q1 = "create table if not exists map_insurance(states varchar(255), years int, quarter int, districts varchar(255), transaction_count bigint, transaction_amount bigint)"
cursor.execute(mi_create_q1)
connection.commit()

mi_insert_q1 = "insert into map_insurance(states, years, quarter, districts, transaction_count, transaction_amount) values (%s,%s,%s,%s,%s,%s)"
values = map_insurance.values.tolist()
cursor.executemany(mi_insert_q1,values)
connection.commit()  


In [13]:
import os
import json
import pandas as pd
import psycopg2

#map transaction 

columns5 = {"States":[],"Years":[],"Quarter":[],"Districts":[],"Transaction_Count":[],"Transaction_Amount":[]}

path5 = "C:/Users/Knodez Guest/Desktop/PhonePe(P)/pulse/data/map/transaction/hover/country/india/state/"
map_state_list = os.listdir(path5)

for state in map_state_list:
    state_path = path5 + state + "/"
    map_year_path = os.listdir(state_path)

    for year in map_year_path:
        year_path = state_path + year + "/"
        map_file_path = os.listdir(year_path)
        
        for file in map_file_path:
            file_path = year_path + file
            
            data = open(file_path,"r")
            E = json.load(data)
            
            for i in E["data"]["hoverDataList"]:
                name = i["name"]
                count = i["metric"][0]["count"]
                amount = i["metric"][0]["amount"]
                columns5["States"].append(state)
                columns5["Years"].append(year)
                columns5["Quarter"].append(int(file.strip(".json")))
                columns5["Districts"].append(name)
                columns5["Transaction_Count"].append(count)
                columns5["Transaction_Amount"].append(amount)

map_transaction = pd.DataFrame(columns5)

map_transaction["States"] = map_transaction["States"].str.replace("-"," ")
map_transaction["States"] = map_transaction["States"].str.title()
map_transaction["Districts"] = map_transaction["Districts"].str.replace("-"," ")
map_transaction["Districts"] = map_transaction["Districts"].str.title()

connection = psycopg2.connect(host="localhost",port="5432",user="postgres",password="12345",database="PhonePe_DB")
cursor = connection.cursor()

mt_create_q1 = "create table if not exists map_transaction(states varchar(255), years int, quarter int, districts varchar(255), transaction_count bigint, transaction_amount bigint)"
cursor.execute(mt_create_q1)
connection.commit()

mt_insert_q1 = "insert into map_transaction(states, years, quarter, districts, transaction_count, transaction_amount) values (%s,%s,%s,%s,%s,%s)"
values = map_transaction.values.tolist()
cursor.executemany(mt_insert_q1,values)
connection.commit()   
            

In [ ]:
import os
import json
import pandas as pd
import psycopg2

#map user 

columns6 = {"States":[],"Years":[],"Quarter":[],"Districts":[],"Registered_Users":[],"App_Opens":[]}

path6 = "C:/Users/Knodez Guest/Desktop/PhonePe(P)/pulse/data/map/user/hover/country/india/state/"
map_state_list = os.listdir(path6)

for state in map_state_list:
    state_path = path6 + state + "/"
    map_year_path = os.listdir(state_path)

    for year in map_year_path:
        year_path = state_path + year + "/"
        map_file_path = os.listdir(year_path)

        for file in map_file_path:
            file_path = year_path + file

            data = open(file_path, "r")
            F = json.load(data)
            
            for i in F["data"]["hoverData"].items():
                districts=i[0]
                registeredUsers=i[1]["registeredUsers"]
                appOpens=i[1]["appOpens"]
                columns6["States"].append(state)
                columns6["Years"].append(year)
                columns6["Quarter"].append(int(file.strip(".json")))
                columns6["Districts"].append(districts)
                columns6["Registered_Users"].append(registeredUsers)
                columns6["App_Opens"].append(appOpens)

map_user = pd.DataFrame(columns6)

map_user["States"] = map_user["States"].str.replace("-"," ")
map_user["States"] = map_user["States"].str.title()
map_user["Districts"] = map_user["Districts"].str.replace("-"," ")
map_user["Districts"] = map_user["Districts"].str.title()

connection = psycopg2.connect(host="localhost",port="5432",user="postgres",password="12345",database="PhonePe_DB")
cursor = connection.cursor()

mu_create_q1 = "create table if not exists map_user(states varchar(255), years int, quarter int, districts varchar(255), registered_users bigint, app_opens bigint)"
cursor.execute(mu_create_q1)
connection.commit()

mu_insert_q1 = "insert into map_user(states, years, quarter, districts, registered_users, app_opens) values (%s,%s,%s,%s,%s,%s)"
values = map_user.values.tolist()
cursor.executemany(mu_insert_q1,values)
connection.commit() 


In [15]:
import os
import json
import pandas as pd
import psycopg2

#top insurnace

columns7 = {"States":[],"Years":[],"Quarter":[],"Pincodes":[],"Transaction_Count":[],"Transaction_Amount":[]}

path7 = "C:/Users/Knodez Guest/Desktop/PhonePe(P)/pulse/data/top/insurance/country/india/state/"
top_state_list = os.listdir(path7)

for state in top_state_list:
    state_path = path7 + state + "/"
    top_year_path = os.listdir(state_path)
    
    for year in top_year_path:
        year_path = state_path + year + "/"
        top_file_path = os.listdir(year_path)
        
        for file in top_file_path:
            file_path = year_path + file
            
            data = open(file_path,"r")
            G = json.load(data)

            for i in G["data"]["pincodes"]:
                entityName = i["entityName"]
                count = i["metric"]["count"]
                amount = i["metric"]["amount"]
                columns7["States"].append(state)
                columns7["Years"].append(year)
                columns7["Quarter"].append(int(file.strip(".json")))
                columns7["Pincodes"].append(entityName)
                columns7["Transaction_Count"].append(count)
                columns7["Transaction_Amount"].append(amount)

top_insurance = pd.DataFrame(columns7)

top_insurance["States"] = top_insurance["States"].str.replace("-"," ")
top_insurance["States"] = top_insurance["States"].str.title()

connection = psycopg2.connect(host="localhost",port="5432",user="postgres",password="12345",database="PhonePe_DB")
cursor = connection.cursor()

ti_create_q1 = "create table if not exists top_insurance(states varchar(255), years int, quarter int, pincodes bigint, transaction_count bigint, transaction_amount bigint)"
cursor.execute(ti_create_q1)
connection.commit()

ti_insert_q1 = "insert into top_insurance(states, years, quarter, pincodes, transaction_count, transaction_amount) values (%s,%s,%s,%s,%s,%s)"
values = top_insurance.values.tolist()
cursor.executemany(ti_insert_q1, values)
connection.commit()

In [16]:
import os
import json
import pandas as pd
import psycopg2

#top_transaction

columns8 = {"States":[],"Years":[],"Quarter":[],"Pincodes":[],"Transaction_Count":[],"Transaction_Amount":[]}

path8 = "C:/Users/Knodez Guest/Desktop/PhonePe(P)/pulse/data/top/transaction/country/india/state/"

top_state_list = os.listdir(path8)

for state in top_state_list:
    state_path = path8 + state + "/"
    top_year_list = os.listdir(state_path)

    for year in top_year_list:
        year_path = state_path + year + "/"
        top_file_list = os.listdir(year_path)

        for file in top_file_list:
            file_path = year_path + file 

            data = open(file_path, "r")
            H = json.load(data)

            for i in H["data"]["pincodes"]:
                entityName = i["entityName"]
                count = i["metric"]["count"]
                amount = i["metric"]["amount"]
                columns8["States"].append(state)
                columns8["Years"].append(year)
                columns8["Quarter"].append(int(file.strip(".json")))
                columns8["Pincodes"].append(entityName)
                columns8["Transaction_Count"].append(count)
                columns8["Transaction_Amount"].append(amount)

top_transaction = pd.DataFrame(columns8)

top_transaction["States"] = top_transaction["States"].str.replace("-"," ")
top_transaction["States"] = top_transaction["States"].str.title()

connection = psycopg2.connect(host="localhost",port="5432",user="postgres",password="12345",database="PhonePe_DB")
cursor = connection.cursor()

tt_create_q1 = "create table if not exists top_transaction(states varchar(255), years int, quarter int, pincodes bigint, transaction_count bigint, transaction_amount bigint)"
cursor.execute(tt_create_q1)
connection.commit()

tt_insert_q1 = "insert into top_transaction(states, years, quarter, pincodes, transaction_count, transaction_amount) values (%s,%s,%s,%s,%s,%s)"
values = top_transaction.values.tolist()
cursor.executemany(tt_insert_q1, values)
connection.commit()


In [17]:
import os
import json
import pandas as pd
import psycopg2

#top_user

columns9 = {"States":[],"Years":[],"Quarter":[],"Pincodes":[],"Registered_Users":[]}

path9 = "C:/Users/Knodez Guest/Desktop/PhonePe(P)/pulse/data/top/user/country/india/state/"

top_state_list = os.listdir(path9)

for state in top_state_list:
    state_path = path9 + state + "/"
    top_year_list = os.listdir(state_path)

    for year in top_year_list:
        year_path = state_path + year + "/"
        top_file_list = os.listdir(year_path)

        for file in top_file_list:
            file_path = year_path + file 

            data = open(file_path, "r")
            I = json.load(data)

            for i in I["data"]["pincodes"]:
                name = i["name"]
                registeredUsers = i["registeredUsers"]
                columns9["States"].append(state)
                columns9["Years"].append(year)
                columns9["Quarter"].append(int(file.strip(".json")))
                columns9["Pincodes"].append(name)
                columns9["Registered_Users"].append(registeredUsers)

top_user = pd.DataFrame(columns9)

top_user["States"] = top_user["States"].str.replace("-"," ")
top_user["States"] = top_user["States"].str.title()

connection = psycopg2.connect(host="localhost",port="5432",user="postgres",password="12345",database="PhonePe_DB")
cursor = connection.cursor()

tu_create_q1 = "create table if not exists top_user(states varchar(255), years int, quarter int, pincodes bigint, registered_users bigint)"
cursor.execute(tu_create_q1)
connection.commit()

tu_insert_q1 = "insert into top_user(states, years, quarter, pincodes, registered_users) values (%s,%s,%s,%s,%s)"
values = top_user.values.tolist()
cursor.executemany(tu_insert_q1, values)
connection.commit()
